In [6]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 1: Secret Entrance ---</h2><p>The Elves have good news and bad news.</p>
<p>The good news is that they've discovered <a href="https://en.wikipedia.org/wiki/Project_management" target="_blank">project management</a>! This has given them the tools they need to prevent their usual Christmas emergency. For example, they now know that the North Pole decorations need to be finished soon so that other critical tasks can start on time.</p>
<p>The bad news is that they've realized they have a <em>different</em> emergency: according to their resource planning, none of them have any time left to decorate the North Pole!</p>
<p>To save Christmas, the Elves need <em>you</em> to <em>finish decorating the North Pole by December 12th</em>.</p>
<p>Collect stars by solving puzzles.  Two puzzles will be made available on each day; the second puzzle is unlocked when you complete the first.  Each puzzle grants <em class="star">one star</em>. Good luck!</p>
<p>You arrive at the secret entrance to the North Pole base ready to start decorating. Unfortunately, the <em>password</em> seems to have been changed, so you can't get in. A document taped to the wall helpfully explains:</p>
<p>"Due to new security protocols, the password is locked in the safe below. Please see the attached document for the new combination."</p>
<p>The safe has a dial with only an arrow on it; around the dial are the numbers <code>0</code> through <code>99</code> in order. As you turn the dial, it makes a small <em>click</em> noise as it reaches each number.</p>
<p>The attached document (your puzzle input) contains a sequence of <em>rotations</em>, one per line, which tell you how to open the safe. A rotation starts with an <code>L</code> or <code>R</code> which indicates whether the rotation should be to the <em>left</em> (toward lower numbers) or to the <em>right</em> (toward higher numbers). Then, the rotation has a <em>distance</em> value which indicates how many clicks the dial should be rotated in that direction.</p>
<p>So, if the dial were pointing at <code>11</code>, a rotation of <code>R8</code> would cause the dial to point at <code>19</code>. After that, a rotation of <code>L19</code> would cause it to point at <code>0</code>.</p>
<p>Because the dial is a circle, turning the dial <em>left from <code>0</code></em> one click makes it point at <code>99</code>. Similarly, turning the dial <em>right from <code>99</code></em> one click makes it point at <code>0</code>.</p>
<p>So, if the dial were pointing at <code>5</code>, a rotation of <code>L10</code> would cause it to point at <code>95</code>. After that, a rotation of <code>R5</code> could cause it to point at <code>0</code>.</p>
<p>The dial starts by pointing at <code>50</code>.</p>
<p>You could follow the instructions, but your recent required official North Pole secret entrance security training seminar taught you that the safe is actually a decoy. The actual password is <em>the number of times the dial is left pointing at <code>0</code> after any rotation in the sequence</em>.</p>
<p>For example, suppose the attached document contained the following rotations:</p>
<pre><code>L68
L30
R48
L5
R60
L55
L1
L99
R14
L82
</code></pre>
<p>Following these rotations would cause the dial to move as follows:</p>
<ul>
<li>The dial starts by pointing at <code>50</code>.</li>
<li>The dial is rotated <code>L68</code> to point at <code>82</code>.</li>
<li>The dial is rotated <code>L30</code> to point at <code>52</code>.</li>
<li>The dial is rotated <code>R48</code> to point at <code><em>0</em></code>.</li>
<li>The dial is rotated <code>L5</code> to point at <code>95</code>.</li>
<li>The dial is rotated <code>R60</code> to point at <code>55</code>.</li>
<li>The dial is rotated <code>L55</code> to point at <code><em>0</em></code>.</li>
<li>The dial is rotated <code>L1</code> to point at <code>99</code>.</li>
<li>The dial is rotated <code>L99</code> to point at <code><em>0</em></code>.</li>
<li>The dial is rotated <code>R14</code> to point at <code>14</code>.</li>
<li>The dial is rotated <code>L82</code> to point at <code>32</code>.</li>
</ul>
<p>Because the dial points at <code>0</code> a total of three times during this process, the password in this example is <code><em>3</em></code>.</p>
<p>Analyze the rotations in your attached document. <em>What's the actual password to open the door?</em></p>
</article>


In [7]:
tests = [
    {
        "name": "Example",
        "s": """
            L68
            L30
            R48
            L5
            R60
            L55
            L1
            L99
            R14
            L82
        """,
        "expected": 3,
    },
]


def part_I(s: str) -> int:
    directions = re.findall(r"[LR]{1}", s)
    distances = [int(d) for d in re.findall(r"[0-9]+", s)]

    dial = list(range(100))
    pointer = 50
    count = 0

    for direction, distance in zip(directions, distances):
        if direction == "L":
            pointer = (pointer - distance) % 100
        else:
            pointer = (pointer + distance) % 100

        if pointer == 0:
            count += 1

    return count


@test(tests=tests)
def test_part_I(s: str) -> int:
    return part_I(s)


Test Example passed, for test_part_I.
Success


In [8]:
with open("../input/day1.txt") as f:
    puzzle = f.read()

print(f"Part I: {part_I(puzzle)}")

Part I: 1084


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1084</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You're sure that's the right password, but the door won't open. You knock, but nobody answers. You build a snowman while you think.</p>
<p>As you're rolling the snowballs for your snowman, you find another security document that must have fallen into the snow:</p>
<p>"Due to newer security protocols, please use <em>password method <span title="You should have seen the chaos when the Elves overflowed their 32-bit password method counter.">0x434C49434B</span></em> until further notice."</p>
<p>You remember from the training seminar that "method 0x434C49434B" means you're actually supposed to count the number of times <em>any click</em> causes the dial to point at <code>0</code>, regardless of whether it happens during a rotation or at the end of one.</p>
<p>Following the same rotations as in the above example, the dial points at zero a few extra times during its rotations:</p>
<ul>
<li>The dial starts by pointing at <code>50</code>.</li>
<li>The dial is rotated <code>L68</code> to point at <code>82</code>; during this rotation, it points at <code>0</code> <em>once</em>.</li>
<li>The dial is rotated <code>L30</code> to point at <code>52</code>.</li>
<li>The dial is rotated <code>R48</code> to point at <code><em>0</em></code>.</li>
<li>The dial is rotated <code>L5</code> to point at <code>95</code>.</li>
<li>The dial is rotated <code>R60</code> to point at <code>55</code>; during this rotation, it points at <code>0</code> <em>once</em>.</li>
<li>The dial is rotated <code>L55</code> to point at <code><em>0</em></code>.</li>
<li>The dial is rotated <code>L1</code> to point at <code>99</code>.</li>
<li>The dial is rotated <code>L99</code> to point at <code><em>0</em></code>.</li>
<li>The dial is rotated <code>R14</code> to point at <code>14</code>.</li>
<li>The dial is rotated <code>L82</code> to point at <code>32</code>; during this rotation, it points at <code>0</code> <em>once</em>.</li>
</ul>
<p>In this example, the dial points at <code>0</code> three times at the end of a rotation, plus three more times during a rotation. So, in this example, the new password would be <code><em>6</em></code>.</p>
<p>Be careful: if the dial were pointing at <code>50</code>, a single rotation like <code>R1000</code> would cause the dial to point at <code>0</code> ten times before returning back to <code>50</code>!</p>
<p>Using password method 0x434C49434B, <em>what is the password to open the door?</em></p>
</article>


In [9]:
tests = [
    {
        "name": "Example",
        "s": """
            L68
            L30
            R48
            L5
            R60
            L55
            L1
            L99
            R14
            L82
        """,
        "expected": 6,
    },
]


def part_II(s: str) -> int:
    directions = re.findall(r"[LR]{1}", s)
    distances = [int(d) for d in re.findall(r"[0-9]+", s)]

    dial = list(range(100))
    pointer = 50
    count = 0

    for direction, distance in zip(directions, distances):
        if direction == "L":
            if pointer <= distance:
                if pointer != 0:
                    count += 1
                count += (distance - pointer) // 100
            pointer = (pointer - distance) % 100
        else:
            if 100 - pointer <= distance:
                count += 1 + (distance + pointer - 100) // 100
            pointer = (pointer + distance) % 100

    return count


@test(tests=tests)
def test_part_II(s: str) -> int:
    return part_II(s)


Test Example passed, for test_part_II.
Success


<link href="style.css" rel="stylesheet"></link>


In [10]:
print(f"Part II: { part_II(puzzle) }")

Part II: 6475


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>6475</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
